### Introduccion
Implementar un modelo de ML para clasificar si un dominio es legítimo o es Phishing. El phising se basa en la ingeniería social (manipulación de emociones, aprovechamiento de atajos mentales y sesgos cognitivos) para engañar a las víctimas y lograr que estas den información (normalmente credenciales).